In [ ]:
import numpy as np

# Define the size of the grid
n = 10

# Generate random travel times (between 1 and 20 minutes)
np.random.seed(0)  # For reproducibility
T = np.random.randint(1, 21, size=(n * n, n * n))

# Generate random priority values (between 0.5 and 3.0)
P = np.random.uniform(0.5, 3.0, size=(n * n, n * n))

# Ensure the diagonal is zero (no self-loops)
np.fill_diagonal(T, 0)
np.fill_diagonal(P, 0)

print("Travel Time Matrix (T):\n", T)
print("Priority Matrix (P):\n", P)



In [ ]:
def calculate_priorities(queue_lengths, alpha):
    num_zones = queue_lengths.shape[0]
    priorities = np.zeros(num_zones)
    for t in range(queue_lengths.shape[1]):
        priorities += alpha ** (queue_lengths.shape[1] - t - 1) * queue_lengths[:, t]
    return priorities

# Example queue lengths over time for each zone (100 zones, 10 time steps)
queue_lengths = np.random.randint(0, 10, size=(n * n, 10))

# Define the discount factor
alpha = 0.9

# Calculate the priorities
priorities = calculate_priorities(queue_lengths, alpha)
print("Priorities:\n", priorities)



In [12]:
# Set the lambda value which is used as a scaling factor for priorities
lambda_value = 1

# Calculate the Combined Cost Matrix (C) by adding the travel time matrix (T) and the scaled priorities
# priorities.reshape(-1, 1) reshapes the priority vector into a column vector for broadcasting over rows of T
C = T + lambda_value * priorities.reshape(-1, 1)

# Print the firs row of Combined Cost Matrix to verify the results
print("First row of Combined Cost Matrix (C):\n", C[0])

First row of Combined Cost Matrix (C):
 [28.27904798 44.27904798 29.27904798 32.27904798 32.27904798 36.27904798
 38.27904798 48.27904798 47.27904798 33.27904798 35.27904798 41.27904798
 30.27904798 35.27904798 36.27904798 43.27904798 46.27904798 34.27904798
 42.27904798 37.27904798 38.27904798 48.27904798 45.27904798 48.27904798
 34.27904798 44.27904798 44.27904798 29.27904798 47.27904798 32.27904798
 46.27904798 48.27904798 48.27904798 48.27904798 43.27904798 36.27904798
 29.27904798 30.27904798 38.27904798 29.27904798 39.27904798 32.27904798
 40.27904798 47.27904798 31.27904798 29.27904798 29.27904798 33.27904798
 34.27904798 35.27904798 37.27904798 46.27904798 44.27904798 33.27904798
 38.27904798 39.27904798 30.27904798 30.27904798 36.27904798 38.27904798
 32.27904798 35.27904798 40.27904798 43.27904798 47.27904798 29.27904798
 43.27904798 32.27904798 41.27904798 39.27904798 40.27904798 33.27904798
 35.27904798 33.27904798 44.27904798 32.27904798 41.27904798 33.27904798
 37.2790479

In [11]:
import networkx as nx

# Create a directed graph from the combined cost matrix
# Initialize a directed graph (DiGraph) using the combined cost matrix C as the adjacency matrix
G = nx.DiGraph(C)
print("Graph:\n", G)
print("\n")

# Find the shortest path from zone 0 to zone 99
# Compute the shortest path in the graph from node (zone) 0 to node (zone) 99 considering the weights ('weight') on the edges, which represent the combined costs

shortest_path = nx.shortest_path(G, source=0, target=99, weight='weight')  
print("Shortest Path from Zone 0 to Zone 99:\n", shortest_path)  

Graph:
 DiGraph with 100 nodes and 10000 edges


Shortest Path from Zone 0 to Zone 99:
 [0, 99]


In [ ]:
# Function to assign drivers to riders
def assign_drivers_to_riders(drivers, riders, C):
    assignments = []  # Initialize an empty list to store the assignments of drivers to riders

    # Iterate over each rider to find the best driver
    for rider in riders:
        # Find the driver with the minimum combined cost to the rider using a lambda function
        # The lambda function takes each driver 'd' and returns the cost from driver 'd' to the rider
        best_driver = min(drivers, key=lambda d: C[d, rider])
        
        # Append the best driver and the rider as a tuple to the assignments list
        assignments.append((best_driver, rider))
        
        # Remove the assigned driver from the list of available drivers to prevent reassignment
        drivers.remove(best_driver)
    
    # Return the list of assignments
    return assignments

# Example drivers and riders
drivers = list(range(5))  # Drivers in zones 0-4, represented as a list of integers
riders = list(range(5, 10))  # Riders in zones 5-9, represented as a list of integers

# Assign drivers to riders using the defined function
assignments = assign_drivers_to_riders(drivers, riders, C)
print("Driver to Rider Assignments:\n", assignments)  # Print the assignments to the console


In [ ]:
def calculate_price(travel_time, priority, base_rate=1.0):
    # This function calculates the price of a ride based on travel time, priority, and a base rate.
    # The price is computed as the product of the base rate, the sum of 1 and the priority factor, and the travel time.
    # The priority factor adjusts the price based on the importance or urgency of the ride.
    return base_rate * (1 + priority) * travel_time

# Calculate prices for the assignments
prices = [calculate_price(T[d, r], priorities[r]) for d, r in assignments]
# This list comprehension iterates over each driver-rider pair in the 'assignments' list.
# For each pair, it retrieves the travel time 'T[d, r]' and the priority 'priorities[r]' for the rider 'r' and driver 'd'.
# It then calls the 'calculate_price' function to compute the price for each assignment.
print("Prices for each assignment:\n", prices)
# Finally, it prints out the calculated prices for each driver-rider assignment.
